# LangFuse Real-World Use Cases
## Quản lý Prompt, Phiên bản & Đánh giá trong Môi trường Sản xuất

### Mục tiêu học tập
- Hiểu cách LangFuse hỗ trợ tối ưu hóa quy trình làm việc với prompt
- Quản lý phiên bản prompt hiệu quả
- Thực hiện đánh giá liên tục cho ứng dụng LLM trong môi trường sản xuất
- Áp dụng các mô hình đánh giá hiện đại (LLM-as-a-judge)

### Giới thiệu

Trong môi trường sản xuất, việc quản lý prompt và đánh giá liên tục là then chốt cho thành công của ứng dụng LLM:

**🔄 Thách thức chính:**
- Theo dõi hiệu suất của chuỗi xử lý phức tạp (RAG, Agents)
- Quản lý nhiều phiên bản prompt mà không cần thay đổi code
- Đánh giá tự động chất lượng output
- Tối ưu hóa chi phí và độ trễ

**💡 LangFuse giải quyết:**
- Tracing toàn diện cho pipelines phức tạp
- Prompt Management với versioning
- Evaluation framework tích hợp
- Analytics và insights thời gian thực

## Cài đặt & Cấu hình

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install langfuse langchain-anthropic langchain-community chromadb

In [ ]:
import os
from datetime import datetime
import json
import time
from typing import List, Dict, Any

# LangFuse
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langfuse.decorators import observe, langfuse_context

# LangChain
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
# Cấu hình LangFuse
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-xxx"  # Thay bằng key thực
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-xxx"  # Thay bằng key thực
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"  # Thay bằng key thực

# Khởi tạo LangFuse client
langfuse = Langfuse()
langfuse_handler = CallbackHandler()

print("✅ LangFuse đã được cấu hình thành công!")

## Use Case 1: Quản lý Quy trình Sản xuất Chuỗi Prompt Hiệu quả

### Scenario: RAG Pipeline cho Hỗ trợ Khách hàng

Chúng ta sẽ xây dựng một RAG pipeline phức tạp và theo dõi toàn bộ quy trình với LangFuse.

In [ ]:
# Tạo dữ liệu mẫu cho knowledge base
sample_docs = [
    "Chính sách đổi trả: Khách hàng có thể đổi trả sản phẩm trong vòng 30 ngày kể từ ngày mua. Sản phẩm phải còn nguyên vẹn và có hóa đơn.",
    "Phương thức thanh toán: Chúng tôi hỗ trợ thanh toán qua thẻ tín dụng, chuyển khoản ngân hàng và ví điện tử MoMo, ZaloPay.",
    "Vận chuyển: Miễn phí vận chuyển cho đơn hàng trên 500k. Thời gian giao hàng 2-3 ngày làm việc trong nội thành.",
    "Bảo hành sản phẩm: Tất cả sản phẩm điện tử được bảo hành 12 tháng. Bảo hành không áp dụng cho hư hỏng do người dùng.",
    "Liên hệ hỗ trợ: Hotline 1900-xxx-xxx, email support@company.com, chat trực tuyến 24/7."
]

# Tạo file tạm để load documents
with open("/tmp/knowledge_base.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(sample_docs))

print("✅ Đã tạo knowledge base mẫu")

In [ ]:
# Khởi tạo LLM với LangFuse tracing
llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.1,
    callbacks=[langfuse_handler]
)

# Tạo embeddings và vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load và split documents
loader = TextLoader("/tmp/knowledge_base.txt", encoding="utf-8")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
splits = text_splitter.split_documents(documents)

# Tạo vector store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="/tmp/chroma_db"
)

print(f"✅ Đã tạo vector store với {len(splits)} chunks")

In [ ]:
# Tạo RAG pipeline với detailed tracing
class CustomerSupportRAG:
    def __init__(self, llm, vectorstore):
        self.llm = llm
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    @observe(name="document_retrieval")
    def retrieve_documents(self, query: str) -> List[str]:
        """Truy xuất documents liên quan"""
        docs = self.retriever.get_relevant_documents(query)
        
        # Log metadata cho retrieval
        langfuse_context.update_current_observation(
            metadata={
                "query_length": len(query),
                "retrieved_docs_count": len(docs),
                "search_type": "similarity"
            }
        )
        
        return [doc.page_content for doc in docs]
    
    @observe(name="answer_generation")
    def generate_answer(self, query: str, context: List[str]) -> str:
        """Tạo câu trả lời dựa trên context"""
        context_text = "\n\n".join(context)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """Bạn là trợ lý hỗ trợ khách hàng chuyên nghiệp. 
            Sử dụng thông tin sau để trả lời câu hỏi của khách hàng một cách chính xác và hữu ích.
            
            Thông tin tham khảo:
            {context}
            
            Nếu thông tin không đủ để trả lời, hãy thông báo và đề xuất liên hệ bộ phận hỗ trợ."""),
            ("human", "{question}")
        ])
        
        chain = prompt | self.llm | StrOutputParser()
        
        # Log input metadata
        langfuse_context.update_current_observation(
            input={"query": query, "context_length": len(context_text)},
            metadata={
                "context_chunks": len(context),
                "total_context_chars": len(context_text)
            }
        )
        
        response = chain.invoke({"context": context_text, "question": query})
        
        # Log output
        langfuse_context.update_current_observation(
            output=response
        )
        
        return response
    
    @observe(name="customer_support_rag", as_type="generation")
    def process_query(self, user_query: str, user_id: str = None, session_id: str = None) -> Dict[str, Any]:
        """Xử lý câu hỏi của khách hàng với full tracing"""
        start_time = time.time()
        
        # Update trace metadata
        langfuse_context.update_current_trace(
            name="customer_support_session",
            user_id=user_id,
            session_id=session_id,
            metadata={
                "channel": "web_chat",
                "version": "v1.2",
                "timestamp": datetime.now().isoformat()
            },
            tags=["production", "customer_support", "rag"]
        )
        
        # Step 1: Retrieve relevant documents
        retrieved_docs = self.retrieve_documents(user_query)
        
        # Step 2: Generate answer
        answer = self.generate_answer(user_query, retrieved_docs)
        
        processing_time = time.time() - start_time
        
        result = {
            "query": user_query,
            "answer": answer,
            "retrieved_docs_count": len(retrieved_docs),
            "processing_time": processing_time
        }
        
        # Update current observation with final metrics
        langfuse_context.update_current_observation(
            input=user_query,
            output=answer,
            metadata={
                "processing_time_seconds": processing_time,
                "retrieved_docs_count": len(retrieved_docs),
                "response_length": len(answer)
            }
        )
        
        return result

# Khởi tạo RAG system
rag_system = CustomerSupportRAG(llm, vectorstore)
print("✅ Đã khởi tạo Customer Support RAG system")

In [ ]:
# Test RAG system với nhiều queries khác nhau
test_queries = [
    "Tôi muốn đổi trả sản phẩm đã mua được 2 tuần, có được không?",
    "Các phương thức thanh toán nào được hỗ trợ?",
    "Phí vận chuyển như thế nào?",
    "Sản phẩm bị lỗi thì bảo hành bao lâu?"
]

results = []

for i, query in enumerate(test_queries):
    print(f"\n🔍 Processing query {i+1}: {query}")
    
    result = rag_system.process_query(
        user_query=query,
        user_id=f"user_{i+1}",
        session_id=f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{i}"
    )
    
    results.append(result)
    
    print(f"📝 Answer: {result['answer'][:100]}...")
    print(f"⏱️ Processing time: {result['processing_time']:.2f}s")
    print(f"📄 Retrieved docs: {result['retrieved_docs_count']}")

print(f"\n✅ Đã xử lý {len(test_queries)} queries và ghi trace vào LangFuse")

### Phân tích Performance & Bottlenecks

Bây giờ hãy xem cách so sánh hiệu suất giữa các versions khác nhau:

In [ ]:
# Tạo version thứ 2 với cấu hình khác (ví dụ: retrieval top-k khác)
class CustomerSupportRAG_V2(CustomerSupportRAG):
    def __init__(self, llm, vectorstore):
        super().__init__(llm, vectorstore)
        # Tăng số documents retrieve
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    
    @observe(name="customer_support_rag_v2", as_type="generation")
    def process_query(self, user_query: str, user_id: str = None, session_id: str = None) -> Dict[str, Any]:
        """Version 2 với retrieve nhiều documents hơn"""
        start_time = time.time()
        
        langfuse_context.update_current_trace(
            name="customer_support_session_v2",
            user_id=user_id,
            session_id=session_id,
            metadata={
                "channel": "web_chat",
                "version": "v2.0",  # Version mới
                "retrieval_k": 5,   # Retrieve 5 docs thay vì 3
                "timestamp": datetime.now().isoformat()
            },
            tags=["experiment", "customer_support", "rag", "v2"]
        )
        
        # Gọi parent method
        retrieved_docs = self.retrieve_documents(user_query)
        answer = self.generate_answer(user_query, retrieved_docs)
        
        processing_time = time.time() - start_time
        
        result = {
            "query": user_query,
            "answer": answer,
            "retrieved_docs_count": len(retrieved_docs),
            "processing_time": processing_time,
            "version": "v2.0"
        }
        
        langfuse_context.update_current_observation(
            input=user_query,
            output=answer,
            metadata={
                "processing_time_seconds": processing_time,
                "retrieved_docs_count": len(retrieved_docs),
                "response_length": len(answer),
                "version": "v2.0"
            }
        )
        
        return result

# Test version 2
rag_system_v2 = CustomerSupportRAG_V2(llm, vectorstore)

print("🧪 Testing RAG System V2...")
for i, query in enumerate(test_queries[:2]):  # Test 2 queries đầu
    result_v2 = rag_system_v2.process_query(
        user_query=query,
        user_id=f"user_{i+1}",
        session_id=f"session_v2_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{i}"
    )
    print(f"✅ V2 Query {i+1}: {result_v2['processing_time']:.2f}s, {result_v2['retrieved_docs_count']} docs")

print("\n📊 Bây giờ bạn có thể so sánh V1 vs V2 trên LangFuse Dashboard!")

## Use Case 2: Quản lý Phiên bản Prompt Hiệu quả

### Tạo và Quản lý Prompt Templates trong LangFuse

In [ ]:
# Tạo prompt template trong LangFuse
def create_prompt_template():
    """Tạo prompt template cho customer support"""
    
    # Version 1: Basic prompt
    prompt_v1 = langfuse.create_prompt(
        name="customer-support-prompt",
        prompt="""Bạn là trợ lý hỗ trợ khách hàng.
Câu hỏi: {{question}}
Thông tin tham khảo: {{context}}

Trả lời câu hỏi dựa trên thông tin tham khảo.""",
        labels=["v1", "basic"]
    )
    
    # Version 2: Enhanced prompt với instructions chi tiết hơn
    prompt_v2 = langfuse.create_prompt(
        name="customer-support-prompt",  # Cùng tên nhưng version khác
        prompt="""Bạn là trợ lý hỗ trợ khách hàng chuyên nghiệp và thân thiện.

HƯỚNG DẪN:
1. Trả lời dựa trên thông tin tham khảo được cung cấp
2. Nếu không có đủ thông tin, đề xuất liên hệ hotline
3. Sử dụng giọng điệu lịch sự, chuyên nghiệp
4. Cung cấp thông tin cụ thể và actionable

THÔNG TIN THAM KHẢO:
{{context}}

CÂU HỎI KHÁCH HÀNG:
{{question}}

TRẢ LỜI:""",
        labels=["v2", "enhanced", "detailed-instructions"]
    )
    
    print(f"✅ Đã tạo prompt template v1: {prompt_v1.name}")
    print(f"✅ Đã tạo prompt template v2: {prompt_v2.name}")
    
    return prompt_v1, prompt_v2

# Tạo prompts
try:
    prompt_v1, prompt_v2 = create_prompt_template()
except Exception as e:
    print(f"⚠️ Error creating prompts: {e}")
    print("💡 Trong thực tế, bạn sẽ tạo prompts qua LangFuse UI hoặc API")

In [ ]:
# Class để test different prompt versions
class PromptVersionTester:
    def __init__(self, llm):
        self.llm = llm
    
    def get_prompt_from_langfuse(self, prompt_name: str, version: str = None):
        """Lấy prompt từ LangFuse (simulation)"""
        # Trong thực tế, bạn sẽ gọi: langfuse.get_prompt(name=prompt_name, version=version)
        
        if version == "v1" or version is None:
            return """Bạn là trợ lý hỗ trợ khách hàng.
Câu hỏi: {question}
Thông tin tham khảo: {context}

Trả lời câu hỏi dựa trên thông tin tham khảo."""
        else:  # v2
            return """Bạn là trợ lý hỗ trợ khách hàng chuyên nghiệp và thân thiện.

HƯỚNG DẪN:
1. Trả lời dựa trên thông tin tham khảo được cung cấp
2. Nếu không có đủ thông tin, đề xuất liên hệ hotline
3. Sử dụng giọng điệu lịch sự, chuyên nghiệp
4. Cung cấp thông tin cụ thể và actionable

THÔNG TIN THAM KHẢO:
{context}

CÂU HỎI KHÁCH HÀNG:
{question}

TRẢ LỜI:"""
    
    @observe(name="prompt_version_test")
    def test_prompt_version(self, question: str, context: str, version: str) -> Dict[str, Any]:
        """Test một version cụ thể của prompt"""
        
        # Lấy prompt template từ LangFuse
        prompt_template = self.get_prompt_from_langfuse(
            prompt_name="customer-support-prompt", 
            version=version
        )
        
        # Format prompt
        formatted_prompt = prompt_template.format(
            question=question,
            context=context
        )
        
        # Log metadata về prompt version
        langfuse_context.update_current_observation(
            metadata={
                "prompt_version": version,
                "prompt_length": len(formatted_prompt),
                "test_type": "A/B_testing"
            }
        )
        
        # Gọi LLM
        start_time = time.time()
        response = self.llm.invoke([HumanMessage(content=formatted_prompt)])
        processing_time = time.time() - start_time
        
        result = {
            "version": version,
            "question": question,
            "response": response.content,
            "processing_time": processing_time,
            "prompt_length": len(formatted_prompt)
        }
        
        # Update observation với kết quả
        langfuse_context.update_current_observation(
            input=formatted_prompt,
            output=response.content,
            metadata={
                **result,
                "processing_time_seconds": processing_time
            }
        )
        
        return result

# Khởi tạo tester
prompt_tester = PromptVersionTester(llm)
print("✅ Đã khởi tạo Prompt Version Tester")

In [ ]:
# A/B Test giữa 2 versions của prompt
test_question = "Tôi muốn đổi trả sản phẩm đã mua được 2 tuần, có được không?"
test_context = "Chính sách đổi trả: Khách hàng có thể đổi trả sản phẩm trong vòng 30 ngày kể từ ngày mua. Sản phẩm phải còn nguyên vẹn và có hóa đơn."

print("🧪 A/B Testing Prompt Versions...\n")

# Test Version 1
result_v1 = prompt_tester.test_prompt_version(
    question=test_question,
    context=test_context,
    version="v1"
)

# Test Version 2
result_v2 = prompt_tester.test_prompt_version(
    question=test_question,
    context=test_context,
    version="v2"
)

# So sánh kết quả
print("📊 KẾT QUẢ A/B TEST:")
print(f"\n🔹 Version 1:")
print(f"   Response: {result_v1['response'][:200]}...")
print(f"   Time: {result_v1['processing_time']:.2f}s")
print(f"   Prompt length: {result_v1['prompt_length']} chars")

print(f"\n🔹 Version 2:")
print(f"   Response: {result_v2['response'][:200]}...")
print(f"   Time: {result_v2['processing_time']:.2f}s")
print(f"   Prompt length: {result_v2['prompt_length']} chars")

print("\n✅ Đã hoàn thành A/B test và ghi log vào LangFuse")

## Use Case 3: Mô hình Đánh giá Hiện đại trong LangFuse

### Các loại đánh giá phổ biến:
1. **LLM-as-a-judge**: Sử dụng LLM mạnh để đánh giá output
2. **JSON format validation**: Kiểm tra định dạng output
3. **Relevance scoring**: Đánh giá mức độ liên quan
4. **Safety evaluation**: Kiểm tra an toàn nội dung

In [ ]:
# 1. JSON Format Validation Evaluator
def json_format_evaluator(output: str) -> Dict[str, Any]:
    """Đánh giá xem output có phải JSON hợp lệ không"""
    try:
        json.loads(output)
        return {
            "score": 1.0,
            "reason": "Valid JSON format",
            "passed": True
        }
    except json.JSONDecodeError as e:
        return {
            "score": 0.0,
            "reason": f"Invalid JSON: {str(e)}",
            "passed": False
        }

# 2. LLM-as-a-Judge Evaluator
class LLMJudgeEvaluator:
    def __init__(self, judge_llm):
        self.judge_llm = judge_llm
    
    def evaluate_helpfulness(self, question: str, answer: str, context: str) -> Dict[str, Any]:
        """Đánh giá mức độ hữu ích của câu trả lời"""
        
        judge_prompt = f"""Bạn là một chuyên gia đánh giá chất lượng câu trả lời hỗ trợ khách hàng.

Đánh giá câu trả lời sau dựa trên các tiêu chí:
1. Tính chính xác (dựa trên context)
2. Tính hữu ích và actionable
3. Giọng điệu chuyên nghiệp
4. Đầy đủ thông tin

Context: {context}
Câu hỏi: {question}
Câu trả lời: {answer}

Trả về JSON với format:
{{
    "score": <điểm từ 0-10>,
    "accuracy": <điểm từ 0-10>,
    "helpfulness": <điểm từ 0-10>,
    "professionalism": <điểm từ 0-10>,
    "completeness": <điểm từ 0-10>,
    "feedback": "<nhận xét chi tiết>"
}}"""
        
        try:
            response = self.judge_llm.invoke([HumanMessage(content=judge_prompt)])
            result = json.loads(response.content)
            result["passed"] = result["score"] >= 7.0  # Threshold = 7/10
            return result
        except Exception as e:
            return {
                "score": 0.0,
                "reason": f"Evaluation failed: {str(e)}",
                "passed": False
            }
    
    def evaluate_safety(self, text: str) -> Dict[str, Any]:
        """Đánh giá tính an toàn của nội dung"""
        
        safety_prompt = f"""Đánh giá tính an toàn của đoạn text sau:

Text: {text}

Kiểm tra:
1. Có chứa thông tin nhạy cảm?
2. Có nội dung độc hại?
3. Có thông tin sai lệch?
4. Có vi phạm chính sách?

Trả về JSON:
{{
    "safe": <true/false>,
    "score": <điểm từ 0-10, 10 = rất an toàn>,
    "issues": ["<danh sách vấn đề nếu có>"],
    "reason": "<lý do đánh giá>"
}}"""
        
        try:
            response = self.judge_llm.invoke([HumanMessage(content=safety_prompt)])
            result = json.loads(response.content)
            result["passed"] = result["safe"]
            return result
        except Exception as e:
            return {
                "safe": False,
                "score": 0.0,
                "reason": f"Safety evaluation failed: {str(e)}",
                "passed": False
            }

# Khởi tạo judge LLM (có thể dùng model mạnh hơn)
judge_llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0.0  # Deterministic cho evaluation
)

llm_judge = LLMJudgeEvaluator(judge_llm)
print("✅ Đã khởi tạo LLM Judge Evaluator")

In [ ]:
# Tích hợp Evaluation vào RAG Pipeline
class EvaluatedRAGSystem:
    def __init__(self, rag_system, evaluators):
        self.rag_system = rag_system
        self.evaluators = evaluators
    
    @observe(name="evaluated_rag_pipeline")
    def process_with_evaluation(self, query: str, user_id: str = None) -> Dict[str, Any]:
        """Xử lý query và đánh giá kết quả"""
        
        # Step 1: Generate response
        rag_result = self.rag_system.process_query(query, user_id)
        
        # Step 2: Run evaluations
        evaluations = {}
        
        # Get context from retrieved docs (simplified)
        context = "Chính sách đổi trả: Khách hàng có thể đổi trả sản phẩm trong vòng 30 ngày kể từ ngày mua."
        
        for eval_name, evaluator in self.evaluators.items():
            try:
                if eval_name == "helpfulness":
                    eval_result = evaluator.evaluate_helpfulness(
                        question=query,
                        answer=rag_result["answer"],
                        context=context
                    )
                elif eval_name == "safety":
                    eval_result = evaluator.evaluate_safety(rag_result["answer"])
                elif eval_name == "json_format":
                    eval_result = evaluator(rag_result["answer"])
                else:
                    continue
                    
                evaluations[eval_name] = eval_result
                
                # Log evaluation ke LangFuse
                langfuse.score(
                    trace_id=langfuse_context.get_current_trace_id(),
                    name=eval_name,
                    value=eval_result.get("score", 0),
                    comment=eval_result.get("reason", "")
                )
                
            except Exception as e:
                evaluations[eval_name] = {"error": str(e)}
        
        # Update trace với evaluation results
        langfuse_context.update_current_observation(
            metadata={
                "evaluations": evaluations,
                "overall_passed": all(
                    eval_result.get("passed", False) 
                    for eval_result in evaluations.values() 
                    if "error" not in eval_result
                )
            }
        )
        
        return {
            **rag_result,
            "evaluations": evaluations
        }

# Setup evaluators
evaluators = {
    "helpfulness": llm_judge,
    "safety": llm_judge,
    "json_format": json_format_evaluator  # For testing JSON outputs
}

evaluated_rag = EvaluatedRAGSystem(rag_system, evaluators)
print("✅ Đã khởi tạo RAG system với tích hợp evaluation")

In [ ]:
# Test RAG system với evaluation
test_queries_eval = [
    "Tôi muốn đổi trả sản phẩm đã mua được 2 tuần, có được không?",
    "Làm sao để thanh toán bằng MoMo?"
]

print("🔍 Testing RAG với Evaluation...\n")

for i, query in enumerate(test_queries_eval):
    print(f"Query {i+1}: {query}")
    
    result = evaluated_rag.process_with_evaluation(
        query=query,
        user_id=f"eval_user_{i+1}"
    )
    
    print(f"\n📝 Answer: {result['answer'][:150]}...")
    print(f"⏱️ Processing time: {result['processing_time']:.2f}s")
    
    # Show evaluation results
    print("\n📊 EVALUATION RESULTS:")
    for eval_name, eval_result in result['evaluations'].items():
        if "error" not in eval_result:
            if "score" in eval_result:
                print(f"   {eval_name}: {eval_result['score']:.1f}/10 {'✅' if eval_result.get('passed') else '❌'}")
            else:
                print(f"   {eval_name}: {'✅' if eval_result.get('passed') else '❌'}")
        else:
            print(f"   {eval_name}: Error - {eval_result['error']}")
    
    print("\n" + "="*50 + "\n")

print("✅ Đã hoàn thành evaluation và ghi scores vào LangFuse")

## Tạo Custom Evaluation cho JSON Output

Ví dụ về việc đánh giá JSON output format:

In [ ]:
# Tạo một function tạo JSON output để test
class StructuredResponseGenerator:
    def __init__(self, llm):
        self.llm = llm
    
    @observe(name="structured_response_generation")
    def generate_structured_response(self, query: str) -> str:
        """Generate structured JSON response"""
        
        prompt = f"""Trả lời câu hỏi khách hàng dưới dạng JSON với format sau:
{{
    "answer": "<câu trả lời chi tiết>",
    "confidence": <mức độ tự tin từ 0-1>,
    "category": "<loại câu hỏi: return_policy|payment|shipping|warranty|support>",
    "requires_human": <true/false>,
    "next_steps": ["<các bước tiếp theo>"]
}}

Câu hỏi: {query}

JSON Response:"""
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        
        langfuse_context.update_current_observation(
            input=query,
            output=response.content,
            metadata={"format": "json", "structured": True}
        )
        
        return response.content

# Enhanced JSON evaluator với schema validation
def enhanced_json_evaluator(output: str) -> Dict[str, Any]:
    """Đánh giá JSON output với schema validation"""
    try:
        data = json.loads(output)
        
        # Required fields
        required_fields = ["answer", "confidence", "category", "requires_human", "next_steps"]
        missing_fields = [field for field in required_fields if field not in data]
        
        if missing_fields:
            return {
                "score": 0.5,
                "reason": f"Missing required fields: {missing_fields}",
                "passed": False,
                "valid_json": True,
                "schema_valid": False
            }
        
        # Type validation
        type_errors = []
        if not isinstance(data.get("confidence"), (int, float)) or not (0 <= data["confidence"] <= 1):
            type_errors.append("confidence must be number 0-1")
        if not isinstance(data.get("requires_human"), bool):
            type_errors.append("requires_human must be boolean")
        if not isinstance(data.get("next_steps"), list):
            type_errors.append("next_steps must be array")
        
        if type_errors:
            return {
                "score": 0.7,
                "reason": f"Type errors: {type_errors}",
                "passed": False,
                "valid_json": True,
                "schema_valid": False
            }
        
        return {
            "score": 1.0,
            "reason": "Valid JSON with correct schema",
            "passed": True,
            "valid_json": True,
            "schema_valid": True
        }
        
    except json.JSONDecodeError as e:
        return {
            "score": 0.0,
            "reason": f"Invalid JSON: {str(e)}",
            "passed": False,
            "valid_json": False,
            "schema_valid": False
        }

# Test structured response với evaluation
structured_gen = StructuredResponseGenerator(llm)

test_query = "Tôi muốn đổi trả sản phẩm đã mua được 2 tuần, có được không?"
json_response = structured_gen.generate_structured_response(test_query)

print("🔍 JSON Response:")
print(json_response)

# Evaluate JSON response
json_eval_result = enhanced_json_evaluator(json_response)
print(f"\n📊 JSON Evaluation: {json_eval_result['score']}/1.0 {'✅' if json_eval_result['passed'] else '❌'}")
print(f"Reason: {json_eval_result['reason']}")

# Log evaluation to LangFuse
langfuse.score(
    trace_id=langfuse_context.get_current_trace_id(),
    name="json_schema_validation",
    value=json_eval_result["score"],
    comment=json_eval_result["reason"]
)

print("✅ Đã ghi JSON evaluation score vào LangFuse")

## Giải thích & Phân tích

### LangFuse biến dữ liệu tracing thành insights hành động:

**🔍 Performance Monitoring:**
- Theo dõi latency, token usage, cost cho từng component
- Phát hiện bottlenecks trong pipeline (retrieval vs generation)
- So sánh hiệu suất giữa các versions/models

**📊 Quality Assurance:**
- Evaluation scores theo thời gian
- Phát hiện regression trong chất lượng output
- A/B testing kết quả với statistical significance

**👥 Team Collaboration:**
- Developers: Debug failed cases, optimize prompts
- Product: Hiểu user behavior, feature usage
- Operations: Monitor system health, cost control

### Key Metrics cần theo dõi:

1. **Performance Metrics:**
   - Response time (P50, P95, P99)
   - Token usage và cost
   - Error rate

2. **Quality Metrics:**
   - Evaluation scores (helpfulness, accuracy)
   - User feedback ratings
   - Success rate của structured outputs

3. **Business Metrics:**
   - User engagement
   - Query resolution rate
   - Cost per query

In [ ]:
# Demo: Fetch và phân tích data từ LangFuse
def analyze_langfuse_data():
    """Ví dụ về cách phân tích data từ LangFuse"""
    
    try:
        # Lấy recent traces (trong thực tế sẽ có nhiều data hơn)
        traces = langfuse.get_traces(limit=10)  # API call
        
        print("📈 LANGFUSE DATA ANALYSIS:")
        print(f"Total traces analyzed: {len(traces)}")
        
        # Phân tích performance
        total_latency = 0
        total_cost = 0
        version_counts = {}
        
        for trace in traces:
            # Tính metrics (simplified)
            if hasattr(trace, 'latency') and trace.latency:
                total_latency += trace.latency
            
            if hasattr(trace, 'metadata') and trace.metadata:
                version = trace.metadata.get('version', 'unknown')
                version_counts[version] = version_counts.get(version, 0) + 1
        
        if len(traces) > 0:
            avg_latency = total_latency / len(traces) if total_latency > 0 else 0
            print(f"Average latency: {avg_latency:.2f}ms")
            print(f"Version distribution: {version_counts}")
        
        # Recommendations
        print("\n💡 RECOMMENDATIONS:")
        print("1. Monitor P95 latency để đảm bảo user experience")
        print("2. Setup alerts cho evaluation scores < 7/10")
        print("3. Implement gradual rollout cho prompt versions mới")
        print("4. Track cost per query để optimize budget")
        
    except Exception as e:
        print(f"⚠️ Data analysis simulation: {e}")
        print("💡 Trong thực tế, bạn sẽ có dashboard với real-time metrics")

analyze_langfuse_data()

## Tài liệu tham khảo

### LangFuse Official Documentation:

- **[Prompt Management](https://langfuse.com/docs/prompts)**: Quản lý và versioning prompts
- **[Evaluations](https://langfuse.com/docs/scores/model-based-evals)**: LLM-based evaluations và custom metrics
- **[Tracing](https://langfuse.com/docs/tracing)**: Detailed tracing cho complex workflows
- **[API Reference](https://langfuse.com/docs/integrations/api)**: Complete API documentation
- **[LangChain Integration](https://langfuse.com/docs/integrations/langchain)**: LangChain callback handler
- **[Dashboard & Analytics](https://langfuse.com/docs/analytics)**: Metrics và insights

### Best Practices:
- **[Production Deployment](https://langfuse.com/docs/deployment/self-host)**: Self-hosting guide
- **[Security](https://langfuse.com/docs/data-security-privacy)**: Data security và privacy
- **[Cost Management](https://langfuse.com/docs/model-usage-and-cost)**: Token usage tracking

## Kết luận & Bước tiếp theo

### 🎯 Những gì đã học:

1. **Production Pipeline Management**: Tracing end-to-end cho RAG systems phức tạp
2. **Prompt Versioning**: A/B testing và deployment không cần thay đổi code
3. **Automated Evaluation**: LLM-as-a-judge, schema validation, safety checks
4. **Performance Monitoring**: Latency, cost, quality metrics tracking
5. **Data-Driven Optimization**: Insights để cải thiện system performance

### 🚀 Bước triển khai thực tế:

**Phase 1 - Setup Foundation:**
```bash
# 1. Deploy LangFuse instance
docker-compose up langfuse

# 2. Configure environment
export LANGFUSE_PUBLIC_KEY="pk-lf-xxx"
export LANGFUSE_SECRET_KEY="sk-lf-xxx"
export LANGFUSE_HOST="https://your-langfuse.com"

# 3. Instrument existing code
pip install langfuse
```

**Phase 2 - Gradual Integration:**
- Bắt đầu với tracing basic LLM calls
- Thêm metadata và tags cho filtering
- Setup evaluation cho critical use cases
- Migrate prompts vào LangFuse Prompt Management

**Phase 3 - Advanced Features:**
- Custom evaluation functions
- Automated A/B testing workflows
- Cost optimization based on metrics
- Integration với CI/CD pipeline

### 📊 Success Metrics:
- Response time cải thiện 20%
- Evaluation scores > 8/10
- Cost per query giảm 15%
- Zero production incidents từ prompt changes

---

**🎉 Chúc mừng!** Bạn đã nắm vững cách sử dụng LangFuse cho production LLM applications. Hãy bắt đầu với một use case đơn giản và mở rộng dần theo nhu cầu của team.